In [2]:
# poblacion_pandemia_simple.py
# Versión simplificada (sin símbolos técnicos como ± o ≈ en la salida)

from __future__ import annotations
from dataclasses import dataclass, asdict
from typing import List, Optional, Dict, Tuple
import numpy as np
from collections import Counter
import math
import random

SEED = 42
rng = np.random.default_rng(SEED)
random.seed(SEED)

OCCUPATION_CATEGORIES = [
    "healthcare",
    "education",
    "service",
    "office",
    "student",
    "unemployed",
]

OCCUPATION_P = np.array([0.06, 0.07, 0.22, 0.26, 0.24, 0.15])

# Roles internos para healthcare y education
HEALTHCARE_ROLES = ["nurse", "physician", "support"]
HEALTHCARE_ROLES_P = np.array([0.55, 0.15, 0.30])
EDUCATION_ROLES = ["teacher", "staff"]
EDUCATION_ROLES_P = np.array([0.80, 0.20])

# Ingresos
INCOME_LEVELS = ["low", "mid", "high"]
INCOME_P_BASE = np.array([0.35, 0.45, 0.20])

ACCESS_LEVELS = ["low", "mid", "high"]

ACCESS_COND_P = {
    "low":  np.array([0.60, 0.30, 0.10]),
    "mid":  np.array([0.25, 0.50, 0.25]),
    "high": np.array([0.10, 0.30, 0.60]),
}

# Distribuciones para continuos
GAMMA_SHAPE = 2.0
GAMMA_SCALE = 0.5
BETA_A = 2.5
BETA_B = 3.0

MOBILITY_MULT_BY_OCC = {
    "healthcare": 1.15,
    "education": 1.05,
    "service": 1.20,
    "office": 0.95,
    "student": 1.10,
    "unemployed": 0.80,
}
COMPLIANCE_SHIFT_BY_OCC = {
    "healthcare": 0.05,
    "education": 0.03,
    "service": -0.02,
    "office": 0.00,
    "student": -0.03,
    "unemployed": -0.01,
}

# Hogares: distribución de tamaño
HOUSEHOLD_SIZE_P = np.array([0.23, 0.28, 0.22, 0.15, 0.08, 0.04])
HOUSEHOLD_SIZES = np.arange(1, len(HOUSEHOLD_SIZE_P) + 1)


@dataclass
class Person:
    pid: int
    occupation: str
    role: Optional[str]
    income_level: str
    vaccine_access: str
    mobility_rate: float
    compliance: float
    household_id: int
    shift: Optional[str]

    def to_dict(self) -> Dict:
        return asdict(self)


# Funciones de muestreo
def sample_macro_occupation() -> str:
    return rng.choice(OCCUPATION_CATEGORIES, p=OCCUPATION_P)


def refine_role(occupation: str) -> Optional[str]:
    if occupation == "healthcare":
        return rng.choice(HEALTHCARE_ROLES, p=HEALTHCARE_ROLES_P)
    if occupation == "education":
        return rng.choice(EDUCATION_ROLES, p=EDUCATION_ROLES_P)
    return None


def sample_income_level() -> str:
    return rng.choice(INCOME_LEVELS, p=INCOME_P_BASE)


def sample_vaccine_access_conditional(income_level: str) -> str:
    p = ACCESS_COND_P[income_level]
    return rng.choice(ACCESS_LEVELS, p=p)


def sample_mobility_rate(occupation: str) -> float:
    base = rng.gamma(GAMMA_SHAPE, GAMMA_SCALE)
    return float(base * MOBILITY_MULT_BY_OCC.get(occupation, 1.0))


def sample_compliance(occupation: str) -> float:
    base = rng.beta(BETA_A, BETA_B)
    shift = COMPLIANCE_SHIFT_BY_OCC.get(occupation, 0.0)
    val = base + shift
    return float(max(0.0, min(1.0, val)))


def assign_households(N: int) -> List[int]:
    """Genera household_id para N personas, usando tamaños aleatorios 1..6."""
    households = []
    hid = 0
    while len(households) < N:
        size = rng.choice(HOUSEHOLD_SIZES, p=HOUSEHOLD_SIZE_P)
        for _ in range(size):
            households.append(hid)
            if len(households) >= N:
                break
        hid += 1
    return households


def sample_population(
    N: int,
    enforce_nurse_night_exact: bool = True,
    target_nurse_night_share: float = 0.20,
    seed: Optional[int] = None,
) -> List[Person]:
    """Muestrea N personas con rasgos discretos/continuos y hogares.
    - Correlación: vaccine_access condicionado a income_level.
    - Turnos: 20% de enfermeras en turno nocturno (exacto si enforce_nurse_night_exact=True).
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)

    household_ids = assign_households(N)

    people: List[Person] = []
    nurse_indices: List[int] = []

    for i in range(N):
        occ = sample_macro_occupation()
        role = refine_role(occ)
        income = sample_income_level()
        access = sample_vaccine_access_conditional(income)
        mobility = sample_mobility_rate(occ)
        compliance = sample_compliance(occ)
        hid = household_ids[i]

        p = Person(
            pid=i,
            occupation=occ,
            role=role,
            income_level=income,
            vaccine_access=access,
            mobility_rate=mobility,
            compliance=compliance,
            household_id=hid,
            shift=None,
        )
        people.append(p)
        if role == "nurse":
            nurse_indices.append(i)

    # Asignar turnos a enfermeras (20% nocturno)
    if nurse_indices:
        if enforce_nurse_night_exact:
            k = int(round(target_nurse_night_share * len(nurse_indices)))
            night_ids = set(rng.choice(nurse_indices, size=k, replace=False).tolist())
            for idx in nurse_indices:
                people[idx].shift = "night" if idx in night_ids else "day"
        else:
            for idx in nurse_indices:
                people[idx].shift = "night" if rng.random() < target_nurse_night_share else "day"

    return people


def summarize_population(people: List[Person]) -> Dict[str, Dict]:
    occ_counts = Counter([p.occupation for p in people])
    role_counts = Counter([p.role for p in people if p.role is not None])
    income_counts = Counter([p.income_level for p in people])
    access_counts = Counter([p.vaccine_access for p in people])

    nurse = [p for p in people if p.role == "nurse"]
    if nurse:
        night = sum(1 for p in nurse if p.shift == "night")
        nurse_night_share = night / len(nurse)
    else:
        nurse_night_share = float("nan")

    mobility = np.array([p.mobility_rate for p in people])
    compliance = np.array([p.compliance for p in people])

    return {
        "occupations": dict(occ_counts),
        "roles": dict(role_counts),
        "income": dict(income_counts),
        "vaccine_access": dict(access_counts),
        "nurses_total": len(nurse),
        "nurses_night_share": nurse_night_share,
        "mobility_mean_std": (float(mobility.mean()), float(mobility.std(ddof=1))),
        "compliance_mean_std": (float(compliance.mean()), float(compliance.std(ddof=1))),
    }


def validate_nurse_night_share(
    people: List[Person], target: float = 0.20
) -> Tuple[float, Tuple[float, float], int]:
    """Devuelve (p_hat, IC95, n_nurses)."""
    nurse = [p for p in people if p.role == "nurse"]
    n = len(nurse)
    if n == 0:
        return (float("nan"), (float("nan"), float("nan")), 0)
    x = sum(1 for p in nurse if p.shift == "night")
    p_hat = x / n
    z = 1.96
    se = math.sqrt(max(p_hat * (1 - p_hat) / max(n, 1), 1e-12))
    ci = (p_hat - z * se, p_hat + z * se)
    return (p_hat, ci, n)


def main():
    N = 10_000
    people = sample_population(N, enforce_nurse_night_exact=True, target_nurse_night_share=0.20, seed=SEED)
    summary = summarize_population(people)
    p_hat, ci, n_nurses = validate_nurse_night_share(people, target=0.20)

    print("=== RESUMEN DE POBLACION ===")
    print(f"Total personas: {N}")
    print("Ocupaciones:", summary["occupations"])
    print("Roles:", summary["roles"])
    print("Ingresos:", summary["income"])
    print("Acceso a vacunas:", summary["vaccine_access"])
    print(f"Enfermeras totales: {n_nurses}")
    print(f"Proporcion nocturna enfermeras: {p_hat:.3f}  IC95: [{ci[0]:.3f}, {ci[1]:.3f}]")

    mu, sd = summary["mobility_mean_std"]
    print(f"Mobility rate - promedio: {mu:.3f}, desviacion: {sd:.3f}")
    mu, sd = summary["compliance_mean_std"]
    print(f"Compliance - promedio: {mu:.3f}, desviacion: {sd:.3f}")

    low_income = [p for p in people if p.income_level == "low"]
    if low_income:
        low_access_given_low_income = sum(1 for p in low_income if p.vaccine_access == "low") / len(low_income)
        print(f"P(low access | low income) ~ {low_access_given_low_income:.3f}")


if __name__ == "__main__":
    main()


=== RESUMEN DE POBLACION ===
Total personas: 10000
Ocupaciones: {'student': 2443, 'office': 2587, 'unemployed': 1522, 'education': 728, 'service': 2150, 'healthcare': 570}
Roles: {'teacher': 592, 'nurse': 327, 'support': 162, 'staff': 136, 'physician': 81}
Ingresos: {'mid': 4507, 'low': 3499, 'high': 1994}
Acceso a vacunas: {'high': 2695, 'low': 3387, 'mid': 3918}
Enfermeras totales: 327
Proporcion nocturna enfermeras: 0.199  IC95: [0.156, 0.242]
Mobility rate - promedio: 1.033, desviacion: 0.736
Compliance - promedio: 0.444, desviacion: 0.194
P(low access | low income) ~ 0.598


# Respuestas:

## 1.d ¿Cómo representar a trabajadores de la salud vs. maestros sin sesgo?

- Usa **columnas separadas** (encendido/apagado) para cada ocupación: `healthcare`, `education`, `service`, etc.  
- Para diferenciar el **riesgo**, variables neutras como: **cuántas personas atienden**, **tiempo frente al público**, **ventilación**, **uso de equipo de protección**.  

**Resultado:** “salud” y “educación” se guardan como columnas separadas, y cualquier diferencia en resultados viene de la **exposición real**, no de la etiqueta.

---

## 2. ¿Cómo muestrear valores “realistas”?

- Para cosas que **no pueden ser negativas** (p. ej., movilidad), con números **positivos** que suelen estar cerca de un valor típico, con algunas personas más arriba y otras más abajo.
- Para cosas entre **0 y 1** (p. ej., cumplimiento de medidas), con valores **dentro de ese rango**, con mayor probabilidad en la zona intermedia.
- Para **categorías** (p. ej., ingreso, acceso a vacunas), se define la **lista de opciones** y sus **porcentajes**.  
  Si **ingreso** y **acceso** están relacionados, primero **ingreso** y después el **acceso** con probabilidades que dependan de ese ingreso.

### 2.a ¿Cómo valido que el 20% de las enfermeras trabaja de noche?

- Después de crear a la población, **al azar** se escoge a **1 de cada 5** enfermeras y las **marcamos**.  
- Verificando que la **proporción** quede **cerca de 0.20 (20%)**. Si no se ajusta. 
---

## 3. ¿Cómo afectan las correlaciones entre rasgos a la dinámica del brote?

- Cuando se juntan **bajos ingresos** y **poco acceso a vacunas**, se acumula más gente **sin protección** en los **mismos hogares y barrios**.
- Los hogares suelen ser **más grandes** y hay **trabajos cara a cara**, lo que implica **más contactos** diarios.
- Esto crea **“bolsones”** donde el virus circula **rápido** y luego **salta** a otros sectores.

**Hipótesis:** habrá **brotes más concentrados** y **picos más altos** en ciertas zonas.  
Las **medidas focalizadas** (vacunación móvil, pruebas, apoyo económico) **bajan más la transmisión** que repartir todo por igual.

---